In [ ]:
# Download and install TensorFlow Object Detection API
!brew install protobuf
!git clone --depth 1 https://github.com/tensorflow/models
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ../../

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Import the TensorFlow Object Detection API visualisation tools
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Display plots in Jupyter
%matplotlib inline

In [ ]:
def load_image(path):
    print(f'Loading image - {path}')
    if path.startswith('http'):
        return np.array(Image.open(tf.keras.utils.get_file(origin=path)))
    return np.array(Image.open(path)) 

def load_label_map(path):
    print(f'Loading label map - {path}')
    return label_map_util.create_category_index_from_labelmap(path)

def load_model(path):
    print(f'Loading model - {path}')
    return hub.load(path)

def run_model(model, image):
    # Add an axis as model expects a tensor representing a batch of images
    image_tensor = tf.expand_dims(tf.convert_to_tensor(image), 0)
    # TensorFlow Object Detection API expects detections to be numpy arrays
    return {key:value.numpy() for key,value in model(image_tensor).items()}

def show_detections(image, results, label_map):
    viz_utils.visualize_boxes_and_labels_on_image_array(
          image,
          results['detection_boxes'][0],
          (results['detection_classes'][0]).astype(int),
          results['detection_scores'][0],
          label_map,
          use_normalized_coordinates=True,
          min_score_thresh=0.3,
          line_thickness=4)
    
    plt.figure(figsize=(24,32))
    plt.imshow(image)
    plt.imsave('annotated.jpg', image)

In [ ]:
# Test image included in TensorFlow Object Detection API.
IMAGE_PATH = 'models/research/object_detection/test_images/image2.jpg'
LABEL_MAP_PATH = 'models/research/object_detection/data/mscoco_label_map.pbtxt'

# EfficientDet D7 1536x1536.
MODEL_PATH = 'https://tfhub.dev/tensorflow/efficientdet/d7/1'

image = load_image(IMAGE_PATH)
labels = load_label_map(LABEL_MAP_PATH)
model = load_model(MODEL_PATH)

results = run_model(model, image)
show_detections(image.copy(), results, labels)